# Monte Carlo Ranom Forest + Linear Regression

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from time import time
import json
import sys

from thorr.utils import read_config

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, mean_squared_error, r2_score

from sklearn.model_selection import KFold, ShuffleSplit, RepeatedKFold, train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import ElasticNetCV, ElasticNet, LinearRegression
from sklearn.inspection import permutation_importance

from joblib import dump, load

from permetrics.regression import RegressionMetric
from quantile_forest import RandomForestQuantileRegressor

import glob

In [2]:
config_path = Path('/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/03-HAB/.env/hab_config.ini')
config_dict = read_config(config_path)
project_dir = Path(config_dict["project"]["project_dir"])

In [3]:
results_dir = Path('/Volumes/STLP-0800/monte_carlo_rfr_base/')
dev_results_dir = results_dir / 'dev_results'
dev_results_dir.mkdir(parents=True, exist_ok=True)
dev_models_dir = results_dir / 'dev_models'
dev_models_dir.mkdir(parents=True, exist_ok=True)
test_results_dir = results_dir / 'test_results'
test_results_dir.mkdir(parents=True, exist_ok=True)
final_models_dir = results_dir / 'final_models'
final_models_dir.mkdir(parents=True, exist_ok=True)

In [4]:
# global parameters
seed = 1955
test_size = 0.2
list_metrics = ["RMSE", "MAE", "NSE", "R2", "KGE", "MSE", "MAPE"]

# Cross-validation parameters
n_splits = 5
n_repeats = 10

In [5]:
hls_insitu = pd.read_csv(project_dir / 'data/hls_insitu/hls_insitu_chl_a_clean.csv', low_memory=False)
hls_insitu['Date'] = pd.to_datetime(hls_insitu['Date'])
hls_insitu['log_chl_a'] = np.log10(hls_insitu['chl_a'])
hls_insitu['doy'] = hls_insitu['Date'].dt.dayofyear

# hls_insitu.columns

In [6]:
dev, test = train_test_split(hls_insitu, test_size=test_size, random_state=seed)
# test = pd.concat([test, joliet_data]).reset_index(drop=True)

del hls_insitu

In [ ]:
# dev.to_csv('dev_set.csv', index=False)
# test.to_csv('test_set.csv', index=False)

dev = pd.read_csv('dev_set.csv')
test = pd.read_csv('test_set.csv')

In [8]:
test['ReachID'].value_counts()

ReachID
12028    682
12029    632
11563    572
11634    432
6928     428
        ... 
5803       1
9653       1
3359       1
12089      1
5973       1
Name: count, Length: 1412, dtype: int64

In [10]:
hlsl30_model_name = "rfr_hlsl30"

hlsl30_features = [
    # "b01_median",
    "b02_median",
    "b03_median",
    "b04_median",
    # "b05_median",
    "b06_median",
    # "b07_median",
    # "b09_median",
    "doy"
]
hlsl30_target = "log_chl_a"

In [11]:
hlss30_model_name = "rfr_hlss30"

hlss30_features = [
    # "b01_median",
    "b02_median",
    "b03_median",
    "b04_median",
    "b05_median",
    # "b06_median",
    "b07_median",
    # "b08_median",
    "b8a_median",
    "b09_median",
    # "b10_median",
    # "b11_median",
    # "b12_median",
    "doy"
]
hlss30_target = "log_chl_a"

## Development (Dev)
Run the algorithm for the training and validation sets (using the dev set)

In [12]:
def oversample_minority_class(X, y, target_col, target_bins, random_state=None):
    oversampled_data = X.copy()
    oversampled_data[target_col] = y

    oversampling_bins = target_bins
    oversampled_data['bin'] = pd.cut(oversampled_data[target_col], bins=oversampling_bins)
    max_bin_count = round(oversampled_data['bin'].value_counts().max()*2)

    oversampled_data_list = []
    for bin in oversampled_data['bin'].unique():
        bin_data = oversampled_data[oversampled_data['bin'] == bin]
        if len(bin_data) >0:
            oversampled_data_list.append(bin_data.sample(max_bin_count, replace=True, random_state=random_state))
    
    oversampled_data = pd.concat(oversampled_data_list).drop(columns=['bin']).reset_index(drop=True)
    
    return oversampled_data.drop(columns=[target_col]), oversampled_data[target_col]

In [ ]:
# Use cross-validation as a Monte Carlo sampler to determine uncertainty in model predictions

# cv_splitter = RepeatedKFold(n_splits=5, n_repeats=400, random_state=42)
# for i, (train_index, test_index) in enumerate(cv_splitter.split(dev)):
for i in range(2000):
    dev = dev.sample(frac=1, random_state=seed+i).reset_index(drop=True) # shuffle the data for each iteration
    train_index, test_index = train_test_split(np.arange(len(dev)), test_size=test_size, random_state=seed+i)

    # l30_train = hls_insitu.iloc[train_index]
    # s30_train = hls_insitu.iloc[train_index]
    if i+1 >= 0: # set to 0 to run all iterations, or set to a specific number to run a subset of iterations
        l30_train = dev.iloc[train_index]
        s30_train = dev.iloc[train_index]
        
        l30_train = l30_train[l30_train['mission'] == 'l30']
        s30_train = s30_train[s30_train['mission'] == 's30']

        l30_train = l30_train.dropna(subset=hlsl30_features + [hlsl30_target])
        s30_train = s30_train.dropna(subset=hlss30_features + [hlss30_target])

        X_l30_train = l30_train[hlsl30_features]
        y_l30_train = l30_train[hlsl30_target]

        oversampled_X_l30_train, oversampled_y_l30_train = oversample_minority_class(X_l30_train, y_l30_train, hlsl30_target, target_bins=np.linspace(-.72, 2.55, 50))

        X_s30_train = s30_train[hlss30_features]
        y_s30_train = s30_train[hlss30_target]

        oversampled_X_s30_train, oversampled_y_s30_train = oversample_minority_class(X_s30_train, y_s30_train, hlss30_target, target_bins=np.linspace(-.72, 2.55, 50))

        rfr_l30 = RandomForestRegressor(n_estimators=250, #min_samples_split=30
                                        max_features="sqrt"
                                        )
        rfr_s30 = RandomForestRegressor(n_estimators=250, #min_samples_split=30,
                                        max_features="sqrt"
                                        )

        # rfr_l30.fit(oversampled_X_l30_train, oversampled_y_l30_train)
        # rfr_s30.fit(oversampled_X_s30_train, oversampled_y_s30_train)

        rfr_l30.fit(X_l30_train, y_l30_train)
        rfr_s30.fit(X_s30_train, y_s30_train)

        l30_train['pred_log_chl_a'] = rfr_l30.predict(X_l30_train)
        s30_train['pred_log_chl_a'] = rfr_s30.predict(X_s30_train)

        train_results = pd.concat([l30_train, s30_train], ignore_index=True)
        train_results = train_results[['Date', 'ReachID', 'mission', 'chl_a', 'log_chl_a', 'pred_log_chl_a']].copy()
        train_results['model_no'] = i+1

        train_results.to_csv(dev_results_dir / f'monte_carlo_train_{i+1}.csv', index=False)

        del l30_train, s30_train

        l30_val = dev.iloc[test_index]
        s30_val = dev.iloc[test_index]

        l30_val = l30_val[l30_val['mission'] == 'l30']
        s30_val = s30_val[s30_val['mission'] == 's30']

        l30_val = l30_val.dropna(subset=hlsl30_features + [hlsl30_target])
        s30_val = s30_val.dropna(subset=hlss30_features + [hlss30_target])

        X_l30_val = l30_val[hlsl30_features]
        y_l30_val = l30_val[hlsl30_target]

        X_s30_val = s30_val[hlss30_features]
        y_s30_val = s30_val[hlss30_target]

        y_l30_pred = rfr_l30.predict(X_l30_val)
        y_s30_pred = rfr_s30.predict(X_s30_val)

        # Store predictions for uncertainty analysis
        l30_val['pred_log_chl_a'] = y_l30_pred
        s30_val['pred_log_chl_a'] = y_s30_pred

        val_results = pd.concat([l30_val, s30_val], ignore_index=True)
        val_results = val_results[['Date', 'ReachID', 'mission', 'chl_a', 'log_chl_a', 'pred_log_chl_a']].copy()
        val_results['model_no'] = i+1

        val_results.to_csv(dev_results_dir / f'monte_carlo_val_{i+1}.csv', index=False)

        del l30_val, s30_val

        dump(rfr_l30, dev_models_dir / f'rfr_l30_model_{i+1}.joblib')
        dump(rfr_s30, dev_models_dir / f'rfr_s30_model_{i+1}.joblib')

        del train_results, val_results
        del rfr_l30, rfr_s30

    if (i+1) % 100 == 0:
        print(f"Completed {i+1} iterations of Monte Carlo cross-validation.")

Completed 100 iterations of Monte Carlo cross-validation.
Completed 200 iterations of Monte Carlo cross-validation.
Completed 300 iterations of Monte Carlo cross-validation.
Completed 400 iterations of Monte Carlo cross-validation.
Completed 500 iterations of Monte Carlo cross-validation.


## Test
Test the algorithm using the test set and the models saaved from the dev mode

In [12]:
# saved models
dev_models = glob.glob(str(dev_models_dir) + '/*.joblib')
n_dev_models = len(dev_models) # combined number of l30 and s30 models

test_results_list = []

for i, model_file in enumerate(dev_models):
    # check if there is l30 or s30 in the filename
    if 'l30' in model_file:
        model = load(model_file)
        test_data = test[test['mission'] == 'l30']
    elif 's30' in model_file:
        model = load(model_file)
        test_data = test[test['mission'] == 's30']
    
    test_data = test_data.dropna(subset=hlsl30_features + [hlsl30_target]) if 'l30' in model_file else test_data.dropna(subset=hlss30_features + [hlss30_target])

    X_test = test_data[hlsl30_features] if 'l30' in model_file else test_data[hlss30_features]

    y_pred = model.predict(X_test)
    test_data['pred_log_chl_a'] = y_pred
    
    # get model number from filename
    model_no = int(model_file.split('_')[-1].split('.')[0])
    test_data['model_no'] = model_no

    test_results_list.append(test_data[['Date', 'ReachID', 'mission', 'chl_a', 'log_chl_a', 'pred_log_chl_a', 'model_no']])

    if (i+1) % 100 == 0:
        print(f"Completed predictions for {i+1} models.")

test_results_df = pd.concat(test_results_list, ignore_index=True)
test_results_df.to_csv(test_results_dir / 'monte_carlo_test_results.csv', index=False)

Completed predictions for 100 models.
Completed predictions for 200 models.
Completed predictions for 300 models.
Completed predictions for 400 models.
Completed predictions for 500 models.
Completed predictions for 600 models.
Completed predictions for 700 models.
Completed predictions for 800 models.
Completed predictions for 900 models.
Completed predictions for 1000 models.
Completed predictions for 1100 models.
Completed predictions for 1200 models.
Completed predictions for 1300 models.
Completed predictions for 1400 models.
Completed predictions for 1500 models.
Completed predictions for 1600 models.
Completed predictions for 1700 models.
Completed predictions for 1800 models.
Completed predictions for 1900 models.
Completed predictions for 2000 models.
Completed predictions for 2100 models.
Completed predictions for 2200 models.
Completed predictions for 2300 models.
Completed predictions for 2400 models.
Completed predictions for 2500 models.
Completed predictions for 2600 mod